# Understanding User Intent

In this lesson, you will build the first agent of your multi-agent system, which will be the user intent agent.

You'll learn:

- how to give an agent a clear task
- how to define tools that are appropriate for the task
- how use state to save important information

Along the way, you will get experience with basic human-in-the-loop interaction


## Agent Details

<img src="images/entire_solution.png" width="500">

The user intent agent is a goal-oriented, conversational agent that helps the user ideate on the kind of graph to build.
- Input: nothing
- Output: `approved_user_goal`, a dictionary pairing a kind of graph with a description of the purpose of the graph.
- Tools: `set_perceived_user_goal`, `approve_perceived_user_goal`

In [2]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.tools import ToolContext

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [3]:
# --- Define Model Constants for easier use ---
MODEL_GPT_4O = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT_4O)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nOpenAI is ready!")

ModelResponse(id='chatcmpl-CAXlXBy4Cz3BN22aujeLmajw5cpU6', created=1756629111, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_cbf1785567', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready! How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

OpenAI is ready!


##  Define the User Intent Agent

### Agent Instructions

You will define the instructions for the agent one part at a time, then combine them into a final set of instructions.

In [4]:
# define the role and goal for the user intent agent
agent_role_and_goal = """
    You are an expert at knowledge graph use cases. 
    Your primary goal is to help the user come up with a knowledge graph use case.
"""

In [7]:
# give the agent some hints about what to say
agent_conversational_hints = """
    If the user is unsure what to do, make some suggestions based on classic use cases like:
    - social network involving friends, family, or professional relationships
    - logistics network with suppliers, customers, and partners
    - recommendation system with customers, products, and purchase patterns
    - fraud detection over multiple accounts with suspicious patterns of transactions
    - pop-culture graphs with movies, books, or music
"""

In [8]:
# describe what the output should look like
agent_output_definition = """
    A user goal has two components:
    - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
    - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."
"""

In [9]:
# specify the steps the agent should follow
agent_chain_of_thought_directions = """
    Think carefully and collaborate with the user:
    1. Understand the user's goal, which is a kind_of_graph with description
    2. Ask clarifying questions as needed
    3. When you think you understand their goal, use the 'set_perceived_user_goal' tool to record your perception
    4. Present the perceived user goal to the user for confirmation
    5. If the user agrees, use the 'approve_perceived_user_goal' tool to approve the user goal. This will save the goal in state under the 'approved_user_goal' key.
"""


In [10]:
# combine all the instruction components into one complete instruction...
complete_agent_instruction = f"""
{agent_role_and_goal}
{agent_conversational_hints}
{agent_output_definition}
{agent_chain_of_thought_directions}
"""

print(complete_agent_instruction)




    You are an expert at knowledge graph use cases. 
    Your primary goal is to help the user come up with a knowledge graph use case.


    If the user is unsure what to do, make some suggestions based on classic use cases like:
    - social network involving friends, family, or professional relationships
    - logistics network with suppliers, customers, and partners
    - recommendation system with customers, products, and purchase patterns
    - fraud detection over multiple accounts with suspicious patterns of transactions
    - pop-culture graphs with movies, books, or music


    A user goal has two components:
    - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
    - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."


    Think carefully and collaborate with the user:
    1.

### Tool Definition

Using tools to define the user goal helps the agent focus on the requirements. 

Rather than open-ended prose, the user goal is defined with specifc arguments passed to a tool.

In [13]:
# Tool: Set Perceived User Goal
# to encourage collaboration with the user, the first tool only sets the perceived user goal

PERCEIVED_USER_GOAL = "perceived_user_goal"

def set_perceived_user_goal(kind_of_graph: str, graph_description:str, tool_context: ToolContext):
    """Sets the perceived user's goal, including the kind of graph and its description.
    
    Args:
        kind_of_graph: 2-3 word definition of the kind of graph, for example "recent US patents"
        graph_description: a single paragraph description of the graph, summarizing the user's intent
    """
    user_goal_data = {"kind_of_graph": kind_of_graph, "graph_description": graph_description}
    tool_context.state[PERCEIVED_USER_GOAL] = user_goal_data
    return tool_success(PERCEIVED_USER_GOAL, user_goal_data)


In [14]:
# Tool: Approve the perceived user goal
# approval from the user should trigger a call to this tool

APPROVED_USER_GOAL = "approved_user_goal"

def approve_perceived_user_goal(tool_context: ToolContext):
    """Upon approval from user, will record the perceived user goal as the approved user goal.
    
    Only call this tool if the user has explicitly approved the perceived user goal.
    """
    # Trust, but verify. 
    # Require that the perceived goal was set before approving it. 
    # Notice the tool error helps the agent take
    if PERCEIVED_USER_GOAL not in tool_context.state:
        return tool_error("perceived_user_goal not set. Set perceived user goal first, or ask clarifying questions if you are unsure.")
    
    tool_context.state[APPROVED_USER_GOAL] = tool_context.state[PERCEIVED_USER_GOAL]

    return tool_success(APPROVED_USER_GOAL, tool_context.state[APPROVED_USER_GOAL])


In [15]:
# add the tools to a list
user_intent_agent_tools = [set_perceived_user_goal, approve_perceived_user_goal]

### Agent Definition

In [16]:
user_intent_agent = Agent(
    name="user_intent_agent_v1",
    model=llm,
    description="Helps the user ideate on a knowledge graph use case.", # used for delegation
    instruction=complete_agent_instruction,
    tools=user_intent_agent_tools,
)
print(f"Agent '{user_intent_agent.name}' created.")

Agent 'user_intent_agent_v1' created.


### Interact with the Agent

In [18]:
from helper import make_agent_caller


user_intent_caller = await make_agent_caller(user_intent_agent)

### Interact with the Agent

In [19]:
session_start = await user_intent_caller.get_session()
print(f"Session Start: {session_start.state}") # expect this to be empty


Session Start: {}


In [20]:
# We need an async function to await for each conversation
async def run_conversation():
    # start things off by describing your goal
    await user_intent_caller.call("""I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis.""") 

    if PERCEIVED_USER_GOAL not in session_start.state:
        # the LLM may have asked a clarifying question. offer some more details
        await user_intent_caller.call("""I'm concerned about possible manufacturing or supplier issues.""")        

    # Optimistically presume approval.
    await user_intent_caller.call("Approve that goal.", True)

In [21]:
await run_conversation()


>>> User Query: I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis.
<<< Agent Response: To create a Bill of Materials (BOM) graph that supports root-cause analysis, I need to clarify a few points to better understand your requirements:

1. **Target Audience:** Who will be using this graph? Is it for internal use by engineers, supply chain managers, or quality control teams?
   
2. **Scope of Analysis:** Do you plan to focus on specific issues like quality defects, delivery delays, or cost overruns, or is your aim a more comprehensive analysis?

3. **Data Sources and Integration:** Do you have existing data sources or systems that need to integrate into the graph, such as ERP systems, supplier databases, etc.?

4. **Desired Features:** Are there particular features you're interested in, like real-time data updates, alert systems for issues, or historical analysis capabilities?

Let me k

In [23]:
# ... existing code ...

# Interactive Conversation Function
async def run_interactive_conversation():
    """
    Interactive version that allows real-time user input while maintaining
    all existing functionality for goal setting and approval.
    """
    print("🤖 Welcome to the Interactive Knowledge Graph Goal Setting Assistant!")
    print("💡 I'll help you define a knowledge graph use case.")
    print("📝 You can type 'quit' or 'exit' to end the conversation at any time.")
    print("=" * 60)
    
    session = await user_intent_caller.get_session()
    print(f"Session Start: {session.state}")
    
    conversation_active = True
    
    while conversation_active:
        try:
            # Get user input
            user_input = input("\n👤 You: ").strip()
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
                print("\n🤖 Thank you for using the Knowledge Graph Goal Setting Assistant!")
                break
            
            # Skip empty inputs
            if not user_input:
                print("⚠️  Please enter a message or type 'quit' to exit.")
                continue
            
            # Call the agent with user input
            response = await user_intent_caller.call(user_input, verbose=False)
            
            # Check current session state
            current_session = await user_intent_caller.get_session()
            
            # Check if we have an approved goal
            if APPROVED_USER_GOAL in current_session.state:
                print("\n✅ Goal Successfully Approved!")
                print("📋 Final Goal Details:")
                approved_goal = current_session.state[APPROVED_USER_GOAL]
                print(f"   🎯 Kind of Graph: {approved_goal.get('kind_of_graph', 'Not specified')}")
                print(f"   📝 Description: {approved_goal.get('graph_description', 'Not specified')}")
                print("\n🎉 You can now proceed to the next step in your knowledge graph construction!")
                
                # Ask if user wants to continue or start over
                continue_choice = input("\nWould you like to (c)ontinue chatting, (r)eset and start over, or (q)uit? [c/r/q]: ").lower().strip()
                
                if continue_choice == 'q':
                    conversation_active = False
                elif continue_choice == 'r':
                    # Reset session state
                    await reset_session()
                    print("\n🔄 Session reset! Let's start fresh.")
                # If 'c' or anything else, continue the conversation
                
        except KeyboardInterrupt:
            print("\n\n⚠️  Conversation interrupted by user.")
            break
        except Exception as e:
            print(f"\n❌ An error occurred: {str(e)}")
            print("🔄 Please try again or type 'quit' to exit.")
    
    # Show final session state
    final_session = await user_intent_caller.get_session()
    print(f"\nFinal Session State: {final_session.state}")

async def reset_session():
    """Reset the session state to start fresh"""
    # Create a new agent caller with empty state
    global user_intent_caller
    user_intent_caller = await make_agent_caller(user_intent_agent, initial_state={})

# Alternative: Non-interactive version (original functionality preserved)
async def run_predefined_conversation():
    """
    Original predefined conversation for backward compatibility
    """
    session_start = await user_intent_caller.get_session()
    print(f"Session Start: {session_start.state}")
    
    # start things off by describing your goal
    await user_intent_caller.call("""I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis.""") 

    if PERCEIVED_USER_GOAL not in session_start.state:
        # the LLM may have asked a clarifying question. offer some more details
        await user_intent_caller.call("""I'm concerned about possible manufacturing or supplier issues.""")        

    # Optimistically presume approval.
    await user_intent_caller.call("Approve that goal.", True)

# Helper function to choose conversation mode
async def start_conversation(interactive=True):
    """
    Start either interactive or predefined conversation
    
    Args:
        interactive (bool): If True, runs interactive mode. If False, runs predefined conversation.
    """
    if interactive:
        await run_interactive_conversation()
    else:
        await run_predefined_conversation()
        session_end = await user_intent_caller.get_session()
        print(f"Session End: {session_end.state}")

# Run the conversation (choose mode)
print("🚀 Choose conversation mode:")
print("1. Interactive mode (real-time user input)")
print("2. Predefined mode (original functionality)")

mode_choice = input("Enter your choice (1 or 2): ").strip()

if mode_choice == "1":
    await start_conversation(interactive=True)
else:
    print("Running predefined conversation...")
    await start_conversation(interactive=False)

🚀 Choose conversation mode:
1. Interactive mode (real-time user input)
2. Predefined mode (original functionality)
Running predefined conversation...
Session Start: {'approved_user_goal': {'kind_of_graph': 'Bill of Materials', 'graph_description': 'A comprehensive BOM graph that includes all levels from suppliers to finished products, supporting root-cause analysis to identify manufacturing or supplier issues. It should enable tracing of issues through the production chain to pinpoint specific materials, components, or suppliers causing failures or delays, with real-time alerts, analytics, and reporting for thorough troubleshooting.'}}

>>> User Query: I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis.
<<< Agent Response: Your goal for a Bill of Materials (BOM) graph that supports root-cause analysis has already been approved with the following specifications:

- **Kind of Graph:** Bil